# Current Cost Markup to DI Equity Position

This is an excerpt of a longer script that calculates the current cost markup to the direct investment equity position.

This calculation involves revaluing the following from historical cost to current cost:
1. structures & equipment (fixed assets)
2. inventories
3. land


The first item, industry-by-year net fixed assets (obtained using entity-level direct investment data from BEA's DI surveys), are revalued to current cost from historical cost using ratios derived from BEA's Fixed Assets Accounts. Since the necessary values are published, the BEA API can be used to retrieve them (rather than downloading excel files on our website).

To install *beaapi*, visit https://github.com/us-bea/beaapi/releases and download the wheel (.whl) file. Then, open your Anaconda terminal and run the following code (if a new version has been released, update the file name):
```
    pip install Downloads\beaapi-0.0.2-py3-none-any.whl
```

You will also need an API key. To register, visit https://apps.bea.gov/api/signup/ and fill out the registration form on the right hand side of the page. The API key will be emailed to you.

# Begin BEA Example Part 1

## I. Preliminaries

Enter your BEA API key:

In [1]:
beakey = '12345678-2222-3333-4444-000000000099'

Import packages needed for this code:

In [2]:
import numpy as np
import pandas as pd
import beaapi

## II. Import Fixed Assets Data

The *beaapi* ```get_data_set_list``` method returns the names and descriptions of the available table series:

In [3]:
beaapi.get_data_set_list(beakey)

DatasetName                                 DatasetDescription
0                      NIPA                               Standard NIPA tables
1        NIUnderlyingDetail               Standard NI underlying detail tables
2                       MNE                          Multinational Enterprises
3               FixedAssets                       Standard Fixed Assets tables
4                       ITA                International Transactions Accounts
5                       IIP                  International Investment Position
6               InputOutput                                  Input-Output Data
7             IntlServTrade                       International Services Trade
8               IntlServSTA  International Services Supplied Through Affili...
9             GDPbyIndustry                                    GDP by Industry
10                 Regional                                 Regional data sets
11  UnderlyingGDPbyIndustry                         Underlying GDP by Industry
12       APIDatasetMetaData                  Metadata about other API datasets

We will be using data from the Fixed Assets dataset. The ```get_parameter_list``` method returns the parameters for which we will need to pass arguments when retrieving the data:

In [4]:
beaapi.get_parameter_list(beakey, 'FixedAssets')

ParameterName ParameterDataType  \
0     TableName            string   
1          Year           integer   

                              ParameterDescription  ParameterIsRequiredFlag  \
0                  The new Fixed Assets identifier                        1   
1  List of year(s) of data to retrieve (X for All)                        1   

  ParameterDefaultValue  MultipleAcceptedFlag AllValue  
0                                           0           
1                                           1        X

The first parameter is **TableName**. Tables are obtained from BEA API using their table names (IDs), which can be found [here](https://apps.bea.gov/api/_pdf/bea_web_service_api_user_guide.pdf). The tables we will import are from the Fixed Assets accounts:
1. CC net stock of private equipment: ```FAAt301E```
2. CC net stock of private structures:  ```FAAt301S```
3. HC net stock of private structures:  ```FAAt303E```
4. HC net stock of private structures:  ```FAAt303S```

**Year** is the other parameter. Use ```Year='X'``` as the argument for all years, ```Year='YYYY'``` for a single year, or ```Year='YYYY, YYYY, YYYY, ...'``` for multiple. Notice that, for multiple years, the argument is not a list of strings, but rather a flat string with commas between years. 

To retrieve data using the BEA API, use the ```get_data``` method. For the Fixed Assets tables, where the parameters are **TableName** and **Year**, with **TableName** taking the form ```FAAt###X```, the code is as follows:
```
beaapi.get_data(beakey, 'FixedAssets', TableName='FAAt###X', Year='YYYY')
```

In [5]:
ccposequip = beaapi.get_data(beakey, 'FixedAssets', TableName='FAAt301E', Year='2020, 2021, 2022')[['LineNumber', 'LineDescription', 'TimePeriod', 'DataValue']] 
ccposstruct = beaapi.get_data(beakey, 'FixedAssets', TableName='FAAt301S', Year='2020, 2021, 2022')[['LineNumber', 'LineDescription', 'TimePeriod', 'DataValue']]
hcposequip = beaapi.get_data(beakey, 'FixedAssets', TableName='FAAt303E', Year='2020, 2021, 2022')[['LineNumber', 'LineDescription', 'TimePeriod', 'DataValue']]
hcposstruct = beaapi.get_data(beakey, 'FixedAssets', TableName='FAAt303S', Year='2020, 2021, 2022')[['LineNumber', 'LineDescription', 'TimePeriod', 'DataValue']]
# the [['LineNumber', 'LineDescription', 'TimePeriod', 'DataValue']] suffix selects only the columns we need
# to see all of the columns returned by the API call, remove that from the first line and re-run this block

Verify the contents of the 4 DataFrames just created

In [6]:
# Add code here

Each table has a **DataValue** column, though the value represented in it is different in each table. These can be renamed for clarity:

In [7]:
ccposequip.rename({'DataValue':'ccposequip'}, axis=1, inplace=True)
ccposstruct.rename({'DataValue':'ccposstruct'}, axis=1, inplace=True)
hcposequip.rename({'DataValue':'hcposequip'}, axis=1, inplace=True)
hcposstruct.rename({'DataValue':'hcposstruct'}, axis=1, inplace=True)

Verify the new column name in each of the 4 DataFrames

In [8]:
# Add code here

# Begin BEA Example Part 2

Now that there are unique column names for the data values in each table, they can be merged:

In [9]:
# merge CC structures with CC equipment (together, CC fixed asstes):
ccposfa = pd.merge(ccposstruct, ccposequip, on=['LineNumber', 'LineDescription', 'TimePeriod'])
# merge HC structures with CC equipment (together, HC fixed assets):
hcposfa = pd.merge(hcposstruct, hcposequip, on=['LineNumber', 'LineDescription', 'TimePeriod'])
# then, merge CC fixed assets with HC fixed assets:
fa =  pd.merge(ccposfa, hcposfa, on=['LineNumber', 'LineDescription', 'TimePeriod'])

fa.head()

LineNumber                              LineDescription TimePeriod  \
0           1                         Private fixed assets       2020   
1           1                         Private fixed assets       2021   
2           1                         Private fixed assets       2022   
3           2  Agriculture, forestry, fishing, and hunting       2020   
4           2  Agriculture, forestry, fishing, and hunting       2021   

   ccposstruct  ccposequip  hcposstruct  hcposequip  
0     40451974     7531434     21369693     7256768  
1     46883307     8118672     22393204     7415794  
2     52663514     8729954     23503241     7679017  
3       395860      323931       169437      297745  
4       443874      359593       175304      308357

## III. Perform Calculations

Finally, the current cost (CC) position to historical cost (HC) position ratio can be calculated. This is the sum of CC stock of structures and equipment divided by the sum of HC stock of structures and equipment.

Add a new column named `cchcratio` with the result of that calculation.

In [10]:
# Add code here

After dropping the intermediary columns, we are left with an industry-by-year series of CC/HC ratios that can be used to revalue industry-by-year net fixed assets.

Drop the 4 intermediary columns then display the resultant DataFrame

In [11]:
# Add code here